# 1. Введение

Цель этой записной книжки - **подготовить универсальные географические данные** с использованием огромного массива пространственных данных.

* Я очищаю набор данных из 11 миллионов записей, аппроксимируя значения таких параметров, как высота над уровнем моря и т.д.
* Во-вторых, я строю визуализацию созданного набора данных и проверяю, насколько они разумны
* Я объединяю набор данных с информацией о населении и делаю выборку на основе этой информации

В результате я создал большой набор данных, содержащий информацию о местоположении (широта, долгота), географической высоте и плотности, который может быть использован как вспомогательный набор данных для любого другого анализа.

**Большая часть кода была скрыта для ясности. Нажмите "показать", чтобы посмотреть там!**


# 2.библиотеки

In [ ]:
!pip install Basemap

In [ ]:
# Импорт основных библиотек для обработки данных
import numpy as np               # Для числовых операций и работы с массивами
import pandas as pd              # Для работы с табличными данными
import random                    # Для генерации случайных чисел
import os                        # Для взаимодействия с операционной системой

# Импорт библиотек для визуализации данных
import matplotlib.pyplot as plt  # Основная библиотека для построения графиков
import seaborn as sns            # Дополнительная библиотека для более стильных графиков


# Импорт библиотек для работы с географическими данными
from mpl_toolkits.basemap import Basemap  # Для создания картографических проекций и отображения данных на карте

# Импорт математических функций и операций
from math import cos, asin, sqrt  # Тригонометрические и другие математические функции
from numpy import nansum, nanmean # Функции для работы с NaN-значениями (например, сумма и среднее)

# Импорт библиотеки для работы с форматом данных NetCDF
import netCDF4                    # Библиотека для чтения и записи данных в формате NetCDF
 # Магическая функция для отображения графиков прямо в Jupyter Notebook
%matplotlib inline             

Шаг 1: Определение библиотек
Здесь подключаются все необходимые библиотеки для работы с данными, визуализации и математических операций.


Шаг 2: Загрузка и обработка исходного набора данных
Под исходным набором данных понимается список всех местоположений на Земле, который очень велик (~11 миллионов записей).
Решено загрузить все 11 миллионов строк. Это значение может быть изменено ниже по коду.


Шаг 3: Проверка на отсутствующие данные
Здесь проводится анализ всего набора данных на предмет пропущенных записей.
Это важно для понимания полноты и качества данных перед дальнейшим анализом.



In [ ]:
import pandas as pd
import requests
import zipfile
import io

# URL файла данных
url = "http://download.geonames.org/export/dump/allCountries.zip"

# Скачивание файла
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Извлечение содержимого ZIP-архива и загрузка в DataFrame
# Обратите внимание: имя файла внутри архива может отличаться, укажите правильное
df = pd.read_csv(z.open('allCountries.txt'), sep='\t', low_memory=False)  

# Просмотр загруженных данных
print(df.head())



In [ ]:
len(df)

In [ ]:
# Установка параметров для выборки данных
n = 11061987  # Общее количество строк в исходном файле
s = 11061987  # Желаемое количество строк для загрузки

# Создание списка индексов строк для пропуска
skip = sorted(random.sample(range(n), n - s))  # Генерация случайного набора индексов строк, которые будут пропущены

# Путь к файлу данных
df_path = "../input/geonames-database/geonames.csv"

# Чтение данных из CSV файла с пропуском определенных строк
#df = pd.read_csv(df_path, index_col='geonameid', skiprows=skip)  # Загрузка данных с указанием 'geonameid' в качестве индекса и пропуском некоторых строк
df = pd.read_csv(df_path, index_col='geonameid', skiprows=skip, low_memory=False)



In [ ]:
# Определение категориальных переменных
C = (df.dtypes == 'object')  # Создание булевой серии, где True обозначает категориальные переменные
CategoricalVariables = list(C[C].index)  # Получение списка названий категориальных переменных

# Определение числовых переменных
Integer = (df.dtypes == 'int64')   # Создание булевой серии для целочисленных переменных
Float   = (df.dtypes == 'float64') # Создание булевой серии для переменных с плавающей точкой
NumericVariables = list(Integer[Integer].index) + list(Float[Float].index)  # Объединение списков целочисленных и с плавающей точкой переменных

# Расчет процента отсутствующих данных
Missing_Percentage = (df.isnull().sum()).sum()/np.product(df.shape)*100  # Вычисление общего процента пропущенных значений в наборе данных

# Вывод процента пропущенных данных
print("Процент отсутствующих записей: " + str(round(Missing_Percentage,2)) + " %")  # Вывод процента пропущенных данных с округлением до двух десятичных знаков


In [ ]:
# Расчет количества отсутствующих значений по каждой переменной
All_NaN = df.isnull().sum()  # Подсчет общего количества отсутствующих значений для каждой переменной в DataFrame

# Определение общего количества строк в DataFrame
RowsCount = len(df.index)  # Получение количества строк в DataFrame

# Вывод процента отсутствующих значений по каждой переменной
print("Процент отсутствующих записей по переменным: ", format(round(All_NaN/RowsCount * 100,5)))  # Вывод процентного соотношения отсутствующих значений для каждой переменной, округленного до пяти десятичных знаков


Давайте перечислим некоторые решения по очистке данных:

* Коды cc2 и административные коды выше 1 будут удалены
* Я буду оценивать высоту, исходя из окружающих территорий
* Я удалю альтернативные названия"

In [ ]:
df=df.drop(['alternatenames','admin2 code','admin3 code','admin4 code','cc2'], axis=1)

Далее, я оцениваю высоту, опираясь на её ближайших соседей. Мы определяем следующую функцию:

функция distance использует географические координаты двух точек (широту и долготу) для расчета расстояния между ними по поверхности земного шара. Расчет основан на формуле гаверсинуса, которая учитывает кривизну Земли. Выводимое сообщение информирует о приблизительном расстоянии между двумя точками в километрах.

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    # Функция для вычисления расстояния между двумя точками на Земле (в километрах)
    p = 0.017453292519943295  # Коэффициент для преобразования градусов в радианы
    # Вычисление расстояния используя формулу гаверсинуса
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))  # Возвращение расстояния в километрах (радиус Земли ~ 6371 км)

print("Длина примерно: " + format(round(distance(51,14,55,24))) + " километров.")


Это позволяет нам рассчитать расстояние между двумя точками, используя функцию гаверсинуса. Ранее я проверил расстояние между двумя точками с разной широтой и долготой на территории Польши. Как я и ожидал, разница между самыми удаленными точками составляет примерно 800 километров. Функция работает. Я буду использовать ее в дальнейшей части анализа. Пока что простая аппроксимация будет достаточной.

Для приближенного определения высоты я применяю упрощенную сетку широты и долготы, где каждое значение округляется до полного градуса (например, 42.523432 В.д. = ~43 В.д.).

Однако я обнаружил, что такое округление все еще недостаточно. Данные о высоте настолько плохие, что я решил округлять до ближайшего четного числа (например, 42.523432 В.д. = ~42 В.д.).


In [ ]:
def round_up_to_even(f):
    # Функция для округления числа до ближайшего четного числа
    return math.ceil(f / 2.) * 2

# Округление широты до ближайшего четного числа
df["latitude_app"] = df.apply(lambda row: round_up_to_even(row['latitude']),axis=1)

# Округление долготы до ближайшего четного числа
df["longitude_app"] = df.apply(lambda row: round_up_to_even(row['longitude']),axis=1)

# Создание таблицы для высоты с округленными широтой и долготой, и подсчет средней высоты
elevation_table = df[['elevation','latitude_app','longitude_app']].groupby(['latitude_app',
                'longitude_app']).agg({'elevation': lambda x: x.mean(skipna=True)}).sort_values(by=['latitude_app', 
                'longitude_app'], ascending=False).reset_index()

# Объединение исходного DataFrame с таблицей высоты
df = pd.merge(df,  elevation_table,  on =['latitude_app', 'longitude_app'],  how ='inner')

# Вывод процента пропущенных значений в столбце 'elevation'
print("Всё ещё есть пропущенные значения в 'elevation': " + format(round((df[['elevation_y']].isnull().sum()).sum()/np.product(df.shape[0])*100,2))  + " %.")





In [ ]:
# Установка средней высоты над уровнем моря в мире
WorldAverageElevation = 840
df['elevation_y'] = df['elevation_y'].fillna(WorldAverageElevation)  # Замена отсутствующих значений в 'elevation_y' на среднюю высоту

df = df.drop(['elevation_x'], axis=1)  # Удаление столбца 'elevation_x' из DataFrame
df  # В


In [ ]:
# Загрузка данных о кодах стран
ISO = pd.read_csv('../input/alpha-country-codes/Alpha__2_and_3_country_codes.csv', sep=';')

# Удаление пробелов в конце строк в столбце 'Country'
ISO['Country'] = ISO.apply(lambda row: str.rstrip(row['Country']), axis=1)

# Удаление ненужных столбцов и переименование столбца для последующего слияния
ISO_toMerge = ISO.drop(['Alpha-3 code', 'Numeric'], axis=1)  # Удаление столбцов 'Alpha-3 code' и 'Numeric'
ISO_toMerge = ISO_toMerge.rename(columns={"Alpha-2 code": "country code"})  # Переименование столбца для совпадения с ключом в df

# Слияние DataFrame df с ISO_toMerge по ключу 'country code'
df = pd.merge(df, ISO_toMerge, on='country code', how='inner')

df  # Вывод обновленного DataFrame


# 3.Data анализ

для экономии памяти 100.000 записей

In [ ]:
# Выборка случайных 100,000 записей из DataFrame
df_sample = df.sample(n=100000)

# Настройка визуализации карты
plt.figure(1, figsize=(12,6))  # Установка размера фигуры
m1 = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=65, llcrnrlon=-180, urcrnrlon=180, lat_ts=0, resolution='c')  # Создание карты с проекцией Mercator

m1.fillcontinents(color='#191919', lake_color='#000000')  # Закраска континентов и озер
m1.drawmapboundary(fill_color='#000000')  # Установка цвета границ карты
m1.drawcountries(linewidth=0.2, color="w")  # Отрисовка границ стран

# Отображение данных на карте
mxy = m1(df_sample["longitude"].tolist(), df_sample["latitude"].tolist())  # Преобразование координат для карты
m1.scatter(mxy[0], mxy[1], s=3, c="#1292db", lw=0, alpha=1, zorder=5)  # Размещение точек на карте

plt.title("Выборка из 100 000 местоположений по всему миру")  # Заголовок графика
plt.show()  # Вывод графика


In [ ]:
# Определение географических границ для Европы
lon_min, lon_max = -10, 40  # Долгота: от -10 до 40
lat_min, lat_max = 35, 65  # Широта: от 35 до 65

# Фильтрация DataFrame для получения записей внутри границ Европы
idx_europe = (df["longitude"] > lon_min) & \
            (df["longitude"] < lon_max) & \
            (df["latitude"] > lat_min) & \
            (df["latitude"] < lat_max)

# Создание нового DataFrame с выборкой из 100,000 местоположений в Европе
df_europe = df[idx_europe].sample(n=100000)

# Настройка визуализации карты Европы
plt.figure(2, figsize=(12,6))  # Установка размера фигуры
m2 = Basemap(projection='merc', llcrnrlat=lat_min, urcrnrlat=lat_max, llcrnrlon=lon_min, urcrnrlon=lon_max, lat_ts=35, resolution='c')  # Создание карты с проекцией Mercator для Европы

m2.fillcontinents(color='#191919', lake_color='#000000')  # Закраска континентов и озер
m2.drawmapboundary(fill_color='#000000')  # Установка цвета границ карты
m2.drawcountries(linewidth=0.2, color="w")  # Отрисовка границ стран

# Отображение данных на карте
mxy = m2(df_europe["longitude"].tolist(), df_europe["latitude"].tolist())  # Преобразование координат для карты
m2.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.05, zorder=5)  # Размещение точек на карте

plt.title("Выборка из 100 000 местоположений в Европе")  # Заголовок графика
plt.show()  # Вывод графика


В этом случае ситуация отличается от ожидаемой. Страны, такие как Великобритания и Бенелюкс, не настолько плотно населены, как предполагалось. Удивительно, но Норвегия, имеющая очень низкую плотность населения, получила большое количество точек. Я вижу, что эти значения тогда не отражают плотность населения.


In [ ]:
# Создание агрегированного DataFrame на основе имени и страны
Aggregated = df[['name', 'Country']]  # Выборка столбцов 'name' и 'Country' из основного DataFrame

# Группировка данных по странам и подсчет количества вхождений
Aggregated = Aggregated.groupby(['Country']).agg(['count']).sort_values([('name', 'count')], ascending=False)

# Расчет процентной доли каждой страны
Aggregated['Percentage'] = round(Aggregated[['name']] / df.shape[0], 2)  # Вычисление процента и округление до двух знаков после запятой

# Переформатирование столбцов для чистоты представления
Aggregated.columns = Aggregated.columns.get_level_values(0)  # Удаление мультиуровневости индексов столбцов
Aggregated.columns = [''.join(col).strip() for col in Aggregated.columns.values]  # Объединение и очистка названий столбцов

Aggregated  # Вывод агрегированного DataFrame


США, Китай, Индия или даже Мексика, находящиеся в топе, вполне логичны. Норвегия же не ожидается в этом списке. У меня есть идея: использовать этот набор данных, но применить выборку для стран на основе их населения. Другими словами, я использую данные точки, но отбираю их с учетом плотности населения. Так, например, в вышеупомянутой таблице, количество точек и в Китае, и в Индии увеличится, в США и Мексике останется высоким, а в Норвегии и других малонаселенных странах сильно уменьшится.

Я объединяю данные о населении с нашими данными ISO, что, конечно, требует некоторых корректировок (из-за различий в названиях стран).


In [ ]:
# Загрузка данных о населении стран
Population = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')

# Переименование столбца для соответствия с другими наборами данных
Population = Population.rename(columns={"Country (or dependency)": "Country"})

# Обновление названий стран для обеспечения согласованности данных
Population[['Country']] = Population[['Country']].replace("Czech Republic (Czechia)", "Czechia")
Population[['Country']] = Population[['Country']].replace("United States", "United States of America")
Population[['Country']] = Population[['Country']].replace("United Kingdom", "United Kingdom of Great Britain and Northern Ireland")
Population[['Country']] = Population[['Country']].replace("Vietnam", "Viet Nam")
Population[['Country']] = Population[['Country']].replace("Laos", "Lao People Democratic Republic")
Population[['Country']] = Population[['Country']].replace("State of Palestine", "Palestine")
Population[['Country']] = Population[['Country']].replace("North Macedonia", "Republic of North Macedonia")
Population[['Country']] = Population[['Country']].replace("Russia", "Russian Federation")
Population[['Country']] = Population[['Country']].replace("Syria", "Syrian Arab Republic")


In [ ]:
# Объединение данных о населении с ISO кодами стран
Population_Merged = pd.merge(ISO_toMerge, Population, on='Country', how='inner')

# Расчет процентной доли населения для каждой страны
Population_Merged[['Population Perc']] = Population_Merged[['Population (2020)']] / Population_Merged[['Population (2020)']].sum()

# Вывод общего процента населения, покрытого объединенными данными
print(format(round(Population_Merged[['Population (2020)']].sum() / Population[['Population (2020)']].sum(), 3)))


 создание взвешенной выборки из основного DataFrame, где веса определяются на основе процентной доли населения каждой страны и их представленности в данных. В результате получается выборка размером в 1,000,000 записей, в которой каждая запись имеет вероятность быть выбранной, пропорциональную размеру выборки страны, к которой она принадлежит.

In [ ]:
# Установка размера выборки
Sample_Size = 1000000

# Объединение с агрегированными данными по странам
Population_Merged = pd.merge(Population_Merged, Aggregated, on='Country', how='inner')

# Расчет размера выборки для каждой страны
Population_Merged['Sample size'] = Population_Merged['Population Perc'] / Population_Merged['name'] * Population_Merged['name'].sum() * Sample_Size

# Создание нового DataFrame с данными о стране и размере выборки
Population_toMerge = Population_Merged.loc[:, Population_Merged.columns.intersection(['Country', 'Sample size'])]

# Объединение основного DataFrame с данными о размере выборки
df = pd.merge(df, Population_toMerge, on='Country', how='inner')

# Расчет общей вероятности выборки
Total_Probability = df['Sample size'].sum()

# Нормализация размеров выборки для каждой записи
df['Sample size'] = df['Sample size'] / Total_Probability

# Создание вектора вероятностей для выборки
vec = df['Sample size']

# Создание выборки с учетом весов (вероятностей) для каждой записи
df_sampled = df.sample(n=Sample_Size, weights='Sample size')


In [ ]:
# Создание агрегированного DataFrame из выборки
Aggregated = df_sampled[['name', 'Country']]  # Выборка столбцов 'name' и 'Country' из выборочного DataFrame
Aggregated = Aggregated.groupby(['Country']).agg(['count']).sort_values([('name', 'count')], ascending=False)  # Группировка и подсчет количества по странам

# Добавление столбца с долей отобранных записей для каждой страны
Aggregated['Sampled records'] = round(Aggregated[['name']] / df_sampled.shape[0], 2)  # Вычисление доли и округление до двух знаков после запятой

# Переформатирование столбцов
Aggregated.columns = Aggregated.columns.get_level_values(0)  # Удаление мультиуровневости индексов столбцов
Aggregated.columns = [''.join(col).strip() for col in Aggregated.columns.values]  # Объединение и очистка названий столбцов

# Создание DataFrame для слияния, содержащего информацию о доле населения каждой страны
Population_toMerge_2 = Population_Merged.loc[:, Population_Merged.columns.intersection(['Country', 'Population Perc'])]

# Объединение агрегированных данных с информацией о доле населения
Aggregated = pd.merge(Aggregated, Population_toMerge_2, on='Country', how='inner')

Aggregated  # Вывод агрегированного DataFrame


In [ ]:
# Определение географических границ для Европы
lat_min, lat_max = 35, 65  # Широта: от 35 до 65

# Фильтрация DataFrame для получения записей внутри границ Европы
idx_europe = (df_sampled["longitude"] > lon_min) & \
            (df_sampled["longitude"] < lon_max) & \
            (df_sampled["latitude"] > lat_min) & \
            (df_sampled["latitude"] < lat_max)

# Создание нового DataFrame с выборкой из 100,000 местоположений в Европе
df_sampled_europe = df_sampled[idx_europe].sample(n=100000)

# Настройка визуализации карты Европы
plt.figure(2, figsize=(12,6))  # Установка размера фигуры
m2 = Basemap(projection='merc', llcrnrlat=lat_min, urcrnrlat=lat_max, llcrnrlon=lon_min, urcrnrlon=lon_max, lat_ts=35, resolution='c')  # Создание карты с проекцией Mercator для Европы

m2.fillcontinents(color='#191919', lake_color='#000000')  # Закраска континентов и озер
m2.drawmapboundary(fill_color='#000000')  # Установка цвета границ карты
m2.drawcountries(linewidth=0.2, color="w")  # Отрисовка границ стран

# Отображение данных на карте
mxy = m2(df_sampled_europe["longitude"].tolist(), df_sampled_europe["latitude"].tolist())  # Преобразование координат для карты
m2.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.05, zorder=5)  # Размещение точек на карте

plt.title("Выборка из 100 000 местоположений в Европе")  # Заголовок графика
plt.show()  # Вывод графика
